In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import os
from utils.common import (
    set_seed,
    m2f_dataset_collate,
    m2f_extract_pred_maps_and_masks,
)
from utils.dataset_utils import (
    get_cadisv2_dataset,
    get_cataract1k_dataset,
    ZEISS_CATEGORIES,
)
from utils.medical_datasets import Mask2FormerDataset
from transformers import (
    Mask2FormerForUniversalSegmentation,
    SwinModel,
    SwinConfig,
    Mask2FormerConfig,
    AutoImageProcessor,
    Mask2FormerImageProcessor
)
from torch.utils.data import DataLoader
import evaluate
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from dotenv import load_dotenv
import wandb

In [3]:
set_seed(42) # seed everything

In [4]:
NUM_CLASSES = len(ZEISS_CATEGORIES) - 3  # Remove class incremental
SWIN_BACKBONE = "microsoft/swin-large-patch4-window12-384"

# Download pretrained swin model
swin_model = SwinModel.from_pretrained(
    SWIN_BACKBONE, out_features=["stage1", "stage2", "stage3", "stage4"]
)
swin_config = SwinConfig.from_pretrained(
    SWIN_BACKBONE, out_features=["stage1", "stage2", "stage3", "stage4"]
)

# Create Mask2Former configuration based on Swin's configuration
mask2former_config = Mask2FormerConfig(
    backbone_config=swin_config, num_labels=NUM_CLASSES #, ignore_value=BG_VALUE
)

# Create the Mask2Former model with this configuration
model = Mask2FormerForUniversalSegmentation(mask2former_config)

# Reuse pretrained parameters
for swin_param, m2f_param in zip(
    swin_model.named_parameters(),
    model.model.pixel_level_module.encoder.named_parameters(),
):
    m2f_param_name = f"model.pixel_level_module.encoder.{m2f_param[0]}"

    if swin_param[0] == m2f_param[0]:
        model.state_dict()[m2f_param_name].copy_(swin_param[1])
        continue

    print(f"Not Matched: {m2f_param[0]} != {swin_param[0]}")

/notebooks/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Not Matched: hidden_states_norms.stage1.weight != layernorm.weight
Not Matched: hidden_states_norms.stage1.bias != layernorm.bias


In [5]:
# Domain incremental datasets
train_cadis_ds, val_cadis_ds, test_cadis_ds = get_cadisv2_dataset(
    "../../storage/data/CaDISv2", domain_incremental=True
)
train_cataract_ds, val_cataract_ds, test_cataract_ds = get_cataract1k_dataset(
    "../../storage/data/cataract-1k", domain_incremental=True
)

# First case: CADIS + CATARACT at the same time
merged_train_ds = torch.utils.data.ConcatDataset([train_cataract_ds, train_cadis_ds])
merged_val_ds = torch.utils.data.ConcatDataset([val_cataract_ds, val_cadis_ds])
merged_test_ds = torch.utils.data.ConcatDataset([test_cataract_ds, test_cadis_ds])

pixel_mean_A,pixel_std_A=pixel_mean_std(train_cadis_ds)
pixel_mean_B,pixel_std_B=pixel_mean_std(merged_train_ds)

# Define preprocessor
swin_processor = AutoImageProcessor.from_pretrained(SWIN_BACKBONE)
m2f_preprocessor_A = Mask2FormerImageProcessor(
    reduce_labels=True,
    ignore_index=255,
    do_resize=False,
    do_rescale=True,
    do_normalize=True,
    #size=#swin_processor.size,
    image_std=pixel_std_A, 
    image_mean=pixel_mean_A 
)

m2f_preprocessor_B = Mask2FormerImageProcessor(
    reduce_labels=True,
    ignore_index=255,
    do_resize=False,
    do_rescale=True,
    do_normalize=True,
    #size=#swin_processor.size,
    image_std=pixel_std_B,
    image_mean=pixel_mean_B
)

# Create M2F Dataset
train_ds = Mask2FormerDataset(merged_train_ds, m2f_preprocessor_B)
val_ds = Mask2FormerDataset(merged_val_ds, m2f_preprocessor_B)
test_ds = Mask2FormerDataset(merged_test_ds, m2f_preprocessor_B)

# Define dataloader params
N_WORKERS = 1
BATCH_SIZE = 2
SHUFFLE = True
DROP_LAST = True

# Define dataloader
train_merged_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=N_WORKERS,
    drop_last=DROP_LAST,
    pin_memory=True,
    collate_fn=m2f_dataset_collate,
)
val_merged_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=N_WORKERS,
    drop_last=DROP_LAST,
    pin_memory=True,
    collate_fn=m2f_dataset_collate,
)
test_merged_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=N_WORKERS,
    drop_last=DROP_LAST,
    pin_memory=True,
    collate_fn=m2f_dataset_collate,
)

/notebooks/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [6]:
# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


In [7]:
# Sanity check :)
#
#for batch in tqdm(train_merged_loader):
#    batch["pixel_values"] = batch["pixel_values"].to(device)
#    batch["pixel_mask"] = batch["pixel_mask"].to(device)
#    batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
#    batch["class_labels"å] = [entry.to(device) for entry in batch["class_labels"]]
#
#for batch in tqdm(val_merged_loader):
#    batch["pixel_values"] = batch["pixel_values"].to(device)
#    batch["pixel_mask"] = batch["pixel_mask"].to(device)
#    batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
#    batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
#  
#for batch in tqdm(test_merged_loader):
#    batch["pixel_values"] = batch["pixel_values"].to(device)
#    batch["pixel_mask"] = batch["pixel_mask"].to(device)
#    batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
#    batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
#

In [7]:
# Training
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
LR_MULTIPLIER = 0.1
BACKBONE_LR = LEARNING_RATE * LR_MULTIPLIER
WEIGHT_DECAY = 0.5
# dice = Dice(average='micro')

# lambda_CE=5.0
# lambda_dice=5.0
metric = evaluate.load("mean_iou")
encoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.encoder")
]
decoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.decoder")
]
transformer_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.transformer_module")
]
optimizer = optim.AdamW(
    [
        {"params": encoder_params, "lr": BACKBONE_LR},
        {"params": decoder_params},
        {"params": transformer_params},
    ],
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

scheduler = optim.lr_scheduler.PolynomialLR(
    optimizer, total_iters=NUM_EPOCHS, power=0.9
)

In [ ]:
# WandB for team usage !!!!

wandb.login() # use this one if a different person is going to run the notebook
#wandb.login(relogin=False) # if the same person in the last run is going to run the notebook again

In [8]:
wandb.init(
    project="M2F_original",
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_multiplier": LR_MULTIPLIER,
        "backbone_learning_rate": BACKBONE_LR,
        "learning_rate_scheduler": scheduler.__class__.__name__,
        "optimizer": optimizer.__class__.__name__,
        "backbone": SWIN_BACKBONE,
        "m2f_preprocessor": m2f_preprocessor.__dict__,
        "m2f_model_config": model.config,
        "num_epochs": NUM_EPOCHS
    },
    name="M2F-Swin-Large-Merged",
    notes="M2F with large Swin backbone pretrained on ImageNet-1K. \
        Scenario: Train, Validate and Test on A and B at the same"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kristiyan-sakalyan (kristiyan-sakalyan-tum). Use `wandb login --relogin` to force relogin


In [9]:
# Tensorboard setup
out_dir="outputs/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(out_dir+"runs"):
    os.makedirs(out_dir+"runs")
%load_ext tensorboard
%tensorboard --logdir outputs/runs

In [10]:
# Tensorboard logging
writer = SummaryWriter(log_dir=out_dir + "runs")

# Model checkpointing
model_name = "m2f_swin_backbone"
model_dir = out_dir + "models/"
if not os.path.exists(model_dir):
    print("Store weights in: ", model_dir)
    os.makedirs(model_dir)

best_model_dir = model_dir + f"{model_name}/best_models/"
if not os.path.exists(model_dir):
    print("Store best model weights in: ", best_model_dir)
    os.makedirs(best_model_dir)
final_model_dir = model_dir + f"{model_name}/final_model/"
if not os.path.exists(model_dir):
    print("Store final model weights in: ", final_model_dir)
    os.makedirs(final_model_dir)

In [12]:
# Save the preprocessor
m2f_preprocessor.save_pretrained(model_dir + model_name)

['outputs/models/m2f_swin_backbone/preprocessor_config.json']

In [13]:
#!rm -r outputs

In [14]:
!CUDA_LAUNCH_BLOCKING=1

In [15]:
# For storing the model
best_val_metric = -np.inf

# Move model to device
model.to(device)

for epoch in range(NUM_EPOCHS):
    model.train()
    train_running_loss = 0.0
    val_running_loss = 0.0

    # Set up tqdm for the training loop
    train_loader = tqdm(
        train_merged_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Training"
    )

    for batch in train_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]

        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss

        # Compute gradient and perform step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        train_running_loss += current_loss
        train_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor
        )
        metric.add_batch(references=masks, predictions=pred_maps)

    # After compute the batches that were added are deleted
    mean_train_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=255, reduce_labels=False
    )["mean_iou"]

    # Validation phase
    model.eval()
    val_loader = tqdm(
        val_merged_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Validation"
    )
    with torch.no_grad():
        for batch in val_loader:
            # Move everything to the device
            batch["pixel_values"] = batch["pixel_values"].to(device)
            batch["pixel_mask"] = batch["pixel_mask"].to(device)
            batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
            batch["class_labels"] = [
                entry.to(device) for entry in batch["class_labels"]
            ]
            # Compute output and loss
            outputs = model(**batch)

            loss = outputs.loss
            # Record losses
            current_loss = loss.item() * batch["pixel_values"].size(0)
            val_running_loss += current_loss
            val_loader.set_postfix(loss=f"{current_loss:.4f}")

            # Extract and compute metrics
            pred_maps, masks = m2f_extract_pred_maps_and_masks(
                batch, outputs, m2f_preprocessor
            )
            metric.add_batch(references=masks, predictions=pred_maps)

    # After compute the batches that were added are deleted
    mean_val_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=255, reduce_labels=False
    )["mean_iou"]

    epoch_train_loss = train_running_loss / len(train_merged_loader.dataset)
    epoch_val_loss = val_running_loss / len(val_merged_loader.dataset)

    writer.add_scalar(f"Loss/train_{model_name}", epoch_train_loss, epoch + 1)
    writer.add_scalar(f"Loss/val_{model_name}", epoch_val_loss, epoch + 1)
    writer.add_scalar(f"mIoU/train_{model_name}", mean_train_iou, epoch + 1)
    writer.add_scalar(f"mIoU/val_{model_name}", mean_val_iou, epoch + 1)

    wandb.log({
        "Loss/train": epoch_train_loss,
        "Loss/val": epoch_val_loss,
        "mIoU/train": mean_train_iou,
        "mIoU/val": mean_val_iou
    })

    if mean_val_iou > best_val_metric:
        best_val_metric = mean_val_iou
        model.save_pretrained(best_model_dir)

    tqdm.write(
        f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_train_loss:.4f}, Train mIoU: {mean_train_iou:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation mIoU: {mean_val_iou:.4f}"
    )

Epoch 1/20 Validation: 100%|██████████| 94/94 [02:19<00:00,  1.48s/it, loss=115.7609]


Epoch 1/20, Train Loss: 36.5734, Train mIoU: 0.1242, Validation Loss: 17.1656, Validation mIoU: 0.2255


Epoch 2/20 Validation: 100%|██████████| 94/94 [02:28<00:00,  1.58s/it, loss=150.3683]


Epoch 2/20, Train Loss: 14.8691, Train mIoU: 0.1979, Validation Loss: 13.7972, Validation mIoU: 0.2754


Epoch 3/20 Validation: 100%|██████████| 94/94 [02:22<00:00,  1.52s/it, loss=77.9847] 


Epoch 3/20, Train Loss: 11.9255, Train mIoU: 0.2336, Validation Loss: 12.6721, Validation mIoU: 0.4001


Epoch 4/20 Validation: 100%|██████████| 94/94 [02:25<00:00,  1.55s/it, loss=80.5599] 


Epoch 4/20, Train Loss: 10.8787, Train mIoU: 0.2644, Validation Loss: 11.4897, Validation mIoU: 0.4380


Epoch 5/20 Training:   0%|          | 1/669 [00:10<1:54:01, 10.24s/it, loss=88.2779]

KeyboardInterrupt



In [11]:
# Load best model and evaluate on test
model = Mask2FormerForUniversalSegmentation.from_pretrained(best_model_dir).to(device)

In [12]:
model.eval()
test_running_loss = 0
test_loader = tqdm(test_merged_loader, desc="Test loop")
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor
        )
        metric.add_batch(references=masks, predictions=pred_maps)

# After compute the batches that were added are deleted
mean_test_iou = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=0, reduce_labels=False
)
final_test_loss = test_running_loss / len(test_merged_loader.dataset)
wandb.log({
    "Loss/test": final_test_loss,
    "mIoU/test": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou}")
wandb.finish()

Test loop: 100%|██████████| 406/406 [08:31<00:00,  1.26s/it, loss=21.9266] 
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


Test Loss: 13.6117, Test mIoU: {'mean_iou': 0.6957735810689049, 'mean_accuracy': 0.8476651174943084, 'overall_accuracy': 0.9544364853220599, 'per_category_iou': array([0.        , 0.89021496, 0.63482789, 0.55386392, 0.55942563,
       0.7503169 , 0.78523856, 0.73548494, 0.888148  , 0.93037863,
       0.92560997]), 'per_category_accuracy': array([       nan, 0.92747735, 0.72603455, 0.78361411, 0.60952089,
       0.80568001, 0.84142026, 0.90333456, 0.95398366, 0.96638072,
       0.95920505])}


In [13]:
mean_test_iou["per_category_iou"]

array([0.        , 0.89021496, 0.63482789, 0.55386392, 0.55942563,
       0.7503169 , 0.78523856, 0.73548494, 0.888148  , 0.93037863,
       0.92560997])